In [1]:
# LLM 에이전트 : LLM이 외부 도구를 활용해서 복잡한 태스크를 수행하는 시스템
# 사용자 요청 -> LLM(추론) ->도구 선택-> 도구 실행 -> 결과 해석 -> 응답

In [2]:
# 에이전트 대화 패턴
# user <-> Assistant
# 사용자가 요청 -> 어시스턴트 응답 ->사용자 피드백 -> ...

# 각 에이전트가 전무 영역을 담당
# Manager -> Agent1 -> Agent2 ->Agent3 ->Mananger

In [3]:
# AutoGen 프레임웍
# 마소에서 개발
# 에이전트가 대화를 통한 협업
# 코드 생성 및 실행 자동화

# AssiatantAgent : 지시에 따라서 작업수행
# UserProxyAgent : 사용자 역활, 코드 실행
# GroupChatManager : 여러 에이진트 조율

In [4]:
%pip install autogen

   ---------------------------------------- 0.0/910.5 kB ? eta -:--:--
   ---------------------------------------- 910.5/910.5 kB 5.7 MB/s  0:00:00

   ---------------------------------------- 0/4 [diskcache]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   ---------- ----------------------------- 1/4 [docker]
   -------------------- ------------------- 2/4 [ag2]
   -------------------- ------------------- 2/4 [ag2]
   -------------------- ------------------- 2/4 [ag2]
  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import autogen
import os
from dotenv import load_dotenv
load_dotenv()
# api 설정
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
]
llm_config = {
    "config_list": config_list,
    "temperature": 0
   
}

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# api 설정
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
]
llm_config = {
    "config_list": config_list,
    "temperature": 0
   
}
# Assistant 에이전트
import autogen
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="너는 친절한 ai 에이전트야"
)

# user proxy 에이전트
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    }
)

# Chat
user_proxy.initiate_chat(
    assistant,
    message="파이썬으로 팩토리얼 함수 작성해줘",
    single_round=True
)

In [ ]:
# 코드 실행 에이전트
import autogen
import os
from dotenv import load_dotenv
load_dotenv()
# api 설정
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
]
llm_config = {
    "config_list": config_list,
    "temperature": 0
   
}
def is_termination_msg(msg):
    content = msg.get("content", "")
    return content and content.strip().endswith('TERMINATE')

# Assistant 에이전트
assistant = autogen.AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
    system_message="너는 친절한 ai 에이전트야"
)
# UserProxy 에이전트
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "workspace",
        "use_docker": False
    }
)
# 타스트 실행
user_proxy.initiate_chat(
    assistant,
    message="삼성전자의 최근 3개월치 주식 데이터를 가져와서 분석하고 시각화 해줘.",
)

In [10]:
# 도구 사용 에이전트
# 커스텀 도구 정의
def search_web(query):
    return f'{query}에 대한 검색결과...'
def calcs(exp):
    return eval(exp)

# Function 등록
import autogen
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

# 함수를 에이전트에 등록
@assistant.register_for_llm(description='web search')
def web_search_tool(query: str) -> str:
    return search_web(query)

@assistant.register_for_llm(description='calculator')
def calculator_tool(expression: str) -> str:
    return str(calcs(expression))

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
)
response1 = assistant.run('calculate 12*(10+3)', user_proxy=user_proxy)
print(response1)

RuntimeError: Code execution is set to be run in docker (default behaviour) but docker is not running.
The options available are:
- Make sure docker is running (advised approach for code execution)
- Set "use_docker": False in code_execution_config
- Set AUTOGEN_USE_DOCKER to "0/False/no" in your environment variables